In [ ]:
import zipfile
import os

# Define paths
zip_file_path = "/content/coat.zip"  # Update with actual filename
extract_path = "/content/coat/"

# Extract the ZIP file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# List extracted files
os.listdir(extract_path)

['coat']

In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# Load COAT dataset (modify paths as needed)
def load_coat_data():
    # Load train (biased) and test (unbiased) data
    train_data = np.loadtxt("/content/coat/coat/train.ascii")  # Shape: [users, items]
    test_data = np.loadtxt("/content/coat/coat/test.ascii")    # Shape: [users, items]

    # Binarize ratings (ratings <4 → 0, else 1)
    train_data = np.where(train_data < 4, 0, 1)
    test_data = np.where(test_data < 4, 0, 1)

    # Split 5% of test data as validation for methods needing unbiased data
    np.random.seed(42)
    unbiased_indices = np.where(test_data != -1)  # Assuming missing entries are marked as -1
    val_indices = np.random.choice(len(unbiased_indices[0]),
                                   size=int(0.05 * len(unbiased_indices[0])),
                                   replace=False)
    val_mask = np.zeros_like(test_data, dtype=bool)
    val_mask[unbiased_indices[0][val_indices], unbiased_indices[1][val_indices]] = True
    test_data[val_mask] = -1  # Mask validation entries in test

    return train_data, test_data, val_mask

train_data, test_data, val_mask = load_coat_data()

In [ ]:
len(train_data)

In [ ]:
class MF(nn.Module):
    def __init__(self, n_users, n_items, latent_dim):
        super().__init__()
        self.user_emb = nn.Embedding(n_users, latent_dim)
        self.item_emb = nn.Embedding(n_items, latent_dim)
        self.sigmoid = nn.Sigmoid()

    def forward(self, user, item):
        user_emb = self.user_emb(user)
        item_emb = self.item_emb(item)
        pred = torch.sum(user_emb * item_emb, dim=1)
        return self.sigmoid(pred)

# Training loop for MF (biased)
def train_mf(model, train_data, epochs=10, lr=0.001):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.BCELoss()

    # Convert data to PyTorch tensors
    users, items = np.where(train_data != -1)
    ratings = train_data[users, items]
    dataset = torch.utils.data.TensorDataset(
        torch.LongTensor(users),
        torch.LongTensor(items),
        torch.FloatTensor(ratings)
    )
    loader = DataLoader(dataset, batch_size=128, shuffle=True)

    # List to store losses
    losses = []

    for epoch in range(epochs):
        epoch_loss = 0.0
        for u, i, r in loader:
            pred = model(u, i)
            loss = criterion(pred, r)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
       # Calculate average epoch loss
        avg_loss = epoch_loss / len(loader)
        losses.append(avg_loss)
        print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}")


    return model

# Initialize and train MF
n_users, n_items = train_data.shape
mf_model = MF(n_users, n_items, latent_dim=64)
train_mf(mf_model, train_data)




Epoch 1/10, Loss: 4.3527
Epoch 2/10, Loss: 3.7052
Epoch 3/10, Loss: 3.1161
Epoch 4/10, Loss: 2.5341
Epoch 5/10, Loss: 1.9639
Epoch 6/10, Loss: 1.4062
Epoch 7/10, Loss: 0.9163
Epoch 8/10, Loss: 0.5561
Epoch 9/10, Loss: 0.3500
Epoch 10/10, Loss: 0.2413


MF(
  (user_emb): Embedding(290, 64)
  (item_emb): Embedding(300, 64)
  (sigmoid): Sigmoid()
)

In [ ]:
class IPS(nn.Module):
    def __init__(self, n_users, n_items, latent_dim):
        super().__init__()
        self.user_emb = nn.Embedding(n_users, latent_dim)
        self.item_emb = nn.Embedding(n_items, latent_dim)
        self.propensity = nn.Sequential(
            nn.Linear(latent_dim * 2, 1),
            nn.Sigmoid()
        )
        self.sigmoid = nn.Sigmoid()

    def forward(self, user, item):
        user_emb = self.user_emb(user)
        item_emb = self.item_emb(item)
        pred = torch.sum(user_emb * item_emb, dim=1)
        return self.sigmoid(pred)

    def train_ips(self, train_data, epochs=10, lr=0.001, eps=1e-8):
        optimizer = torch.optim.Adam(self.parameters(), lr=lr)
        criterion = nn.BCELoss(reduction='none')  # Use reduction='none' to compute δ_{u,i} per sample

        # Create dataset (include all user-item pairs in D)
        users_all, items_all = np.indices(train_data.shape).reshape(2, -1)
        ratings_all = train_data.ravel()  # Flatten the matrix
        o = (ratings_all != -1).astype(float)  # Observation indicator (1 if observed, 0 otherwise)
        dataset = TensorDataset(
            torch.LongTensor(users_all),
            torch.LongTensor(items_all),
            torch.FloatTensor(ratings_all),
            torch.FloatTensor(o)
        )
        loader = DataLoader(dataset, batch_size=128, shuffle=True)

        for epoch in range(epochs):
            epoch_loss = 0.0
            for u, i, r, o_batch in loader:
                # Compute propensity scores p̂_{u,i}
                user_emb = self.user_emb(u)
                item_emb = self.item_emb(i)
                prop_input = torch.cat([user_emb, item_emb], dim=1)
                p_hat = torch.clamp(self.propensity(prop_input).squeeze(), min=eps, max=1-eps)

                # Compute prediction error δ_{u,i}
                pred = self(u, i)
                delta = criterion(pred, r)  # δ_{u,i} = BCE(r_{u,i}, r̂_{u,i})

                # Compute IPS loss: (o_{u,i} * δ_{u,i}) / p̂_{u,i}
                loss = (o_batch * delta / p_hat).mean()

                # Backpropagation
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                epoch_loss += loss.item()

            avg_loss = epoch_loss / len(loader)
            print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}")

# Initialize and train IPS model
ips_model = IPS(n_users=290, n_items=300, latent_dim=64)
ips_model.train_ips(train_data)

Epoch 1/10, Loss: 7.4812
Epoch 2/10, Loss: 4.8321
Epoch 3/10, Loss: 3.5211
Epoch 4/10, Loss: 2.5824
Epoch 5/10, Loss: 1.7830
Epoch 6/10, Loss: 1.1505
Epoch 7/10, Loss: 0.7108
Epoch 8/10, Loss: 0.4510
Epoch 9/10, Loss: 0.3124
Epoch 10/10, Loss: 0.2376


In [ ]:
class DR(IPS):
    def __init__(self, n_users, n_items, latent_dim):
        super().__init__(n_users, n_items, latent_dim)
        # Imputation model for δ̂_u,i
        self.imputation = nn.Sequential(
            nn.Linear(latent_dim * 2, 1),  # Input: user + item embeddings
            nn.Sigmoid()
        )

    def compute_dr_loss(self, u, i, r, eps=1e-8):
        # Get embeddings
        user_emb = self.user_emb(u)
        item_emb = self.item_emb(i)
        prop_input = torch.cat([user_emb, item_emb], dim=1)

        # Predicted conversion rate (CVR)
        pred = self(u, i)  # r̂_u,i

        # Compute true loss δ_u,i (BCE)
        delta = - (r * torch.log(pred + eps) + (1 - r) * torch.log(1 - pred + eps))

        # Compute imputed error δ̂_u,i
        delta_hat = self.imputation(prop_input).squeeze()

        # Compute propensity score p̂_u,i (clamped to avoid division by 0)
        p_hat = torch.clamp(self.propensity(prop_input).squeeze(), min=eps, max=1-eps)

        # Observation indicator o_u,i (1 for observed interactions)
        o = (r != -1).float()  # Assuming unobserved entries are marked with -1

        # DR loss component
        dr_term = delta_hat + (o * (delta - delta_hat)) / p_hat

        return dr_term.mean()  # Average over batch

    def train_dr(self, train_data, epochs=10, lr=0.001, eps=1e-8):
        optimizer = torch.optim.Adam(self.parameters(), lr=lr)
        losses = []

        # Prepare dataset
        users, items = np.where(train_data != -1)
        ratings = train_data[users, items]
        dataset = torch.utils.data.TensorDataset(
            torch.LongTensor(users),
            torch.LongTensor(items),
            torch.FloatTensor(ratings)
        )
        loader = DataLoader(dataset, batch_size=128, shuffle=True)

        for epoch in range(epochs):
            epoch_loss = 0.0
            for u, i, r in loader:
                # Compute DR loss
                loss = self.compute_dr_loss(u, i, r, eps)

                # Backpropagation with gradient clipping
                optimizer.zero_grad()
                loss.backward()
                torch.nn.utils.clip_grad_norm_(self.parameters(), max_norm=1.0)
                optimizer.step()

                epoch_loss += loss.item()

            avg_loss = epoch_loss / len(loader)
            losses.append(avg_loss)
            print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}")

        return self, losses
# Initialize and train the DR model
dr_model = DR(n_users, n_items, latent_dim=64)
trained_model, training_losses = dr_model.train_dr(train_data)

Epoch 1/10, Loss: 4.6169
Epoch 2/10, Loss: 2.9075
Epoch 3/10, Loss: 2.1495
Epoch 4/10, Loss: 1.5732
Epoch 5/10, Loss: 1.0609
Epoch 6/10, Loss: 0.6699
Epoch 7/10, Loss: 0.4265
Epoch 8/10, Loss: 0.2948
Epoch 9/10, Loss: 0.2259
Epoch 10/10, Loss: 0.1866


In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

# ==================== 1. Define Base Components ====================
class ResidualLayer(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.residual = nn.Sequential(
            nn.Linear(input_dim, input_dim),
            nn.ReLU()
        )

    def forward(self, x):
        return x + self.residual(x)

class IPS(nn.Module):
    def __init__(self, n_users, n_items, latent_dim):
        super().__init__()
        self.user_emb = nn.Embedding(n_users, latent_dim)
        self.item_emb = nn.Embedding(n_items, latent_dim)
        self.propensity = nn.Sequential(
            nn.Linear(latent_dim*2, 1),
            nn.Sigmoid()
        )

    def forward(self, user, item):
        user_emb = self.user_emb(user)
        item_emb = self.item_emb(item)
        return torch.sigmoid(torch.sum(user_emb * item_emb, dim=1))

# ==================== 2. Implement Res-IPS ====================
class ResIPS(IPS):
    def __init__(self, n_users, n_items, latent_dim):
        super().__init__(n_users, n_items, latent_dim)
        # Residual networks
        self.residual_p = nn.Sequential(
            ResidualLayer(latent_dim*2),
            nn.Linear(latent_dim*2, 1)
        )
        self.residual_delta = nn.Sequential(
            ResidualLayer(latent_dim*2),
            nn.Linear(latent_dim*2, 1)
        )
        # Imputation model
        self.imputation = nn.Sequential(
            nn.Linear(latent_dim*2, 1),
            nn.Sigmoid()
        )

    def _prepare_data(self, train_data, val_data):
        """Create data loaders for training and validation"""
        # Training data (biased)
        train_users, train_items = np.where(train_data != -1)
        train_ratings = train_data[train_users, train_items]
        train_dataset = TensorDataset(
            torch.LongTensor(train_users),
            torch.LongTensor(train_items),
            torch.FloatTensor(train_ratings)
        )
        train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)

        # Validation data (unbiased)
        val_users, val_items = np.where(val_data != -1)
        val_ratings = val_data[val_users, val_items]
        val_dataset = TensorDataset(
            torch.LongTensor(val_users),
            torch.LongTensor(val_items),
            torch.FloatTensor(val_ratings)
        )
        val_loader = DataLoader(val_dataset, batch_size=128)

        return train_loader, val_loader

    def compute_loss(self, u, i, r, vu, vi, vr, alpha=0.5, beta=1.0, gamma=0.01, eps=1e-8):
        """Calculate complete Res-IPS loss"""
        # Feature concatenation
        user_emb = self.user_emb(u)
        item_emb = self.item_emb(i)
        phi = torch.cat([user_emb, item_emb], dim=1)

        # Propensity calibration
        p_nominal = torch.clamp(self.propensity(phi).squeeze(), min=eps, max=1-eps)
        p_res = self.residual_p(phi).squeeze()
        p_tilde = torch.sigmoid(torch.logit(p_nominal) + p_res).clamp(eps, 1-eps)

        # Imputation calibration
        delta_nominal = self.imputation(phi).squeeze()
        delta_res = self.residual_delta(phi).squeeze()
        delta_tilde = torch.sigmoid(torch.logit(delta_nominal) + delta_res).clamp(eps, 1-eps)

        # Loss components
        L_ctr = F.binary_cross_entropy(p_nominal, r)
        L_imp = F.mse_loss(delta_nominal, (r - self(u,i)).detach())
        L_cvr_b = (F.binary_cross_entropy(self(u,i), r, reduction='none') / p_tilde).mean()
        L_ctcvr_b = F.binary_cross_entropy(p_tilde * self(u,i), r)
        L_cvr_u = F.binary_cross_entropy(self(vu,vi), vr)
        L_consistency = F.mse_loss(p_nominal, p_tilde) + F.mse_loss(delta_nominal, delta_tilde)

        return L_ctr + alpha*L_imp + beta*(L_cvr_b + L_ctcvr_b + L_cvr_u) + gamma*L_consistency

    def train_resips(self, train_data, val_data, epochs=10, lr=0.001, **kwargs):
        optimizer = torch.optim.Adam(self.parameters(), lr=lr)
        train_loader, val_loader = self._prepare_data(train_data, val_data)
        losses = []

        for epoch in range(epochs):
            epoch_loss = 0.0
            for (u,i,r), (vu,vi,vr) in zip(train_loader, val_loader):
                loss = self.compute_loss(u,i,r, vu,vi,vr, **kwargs)
                optimizer.zero_grad()
                loss.backward()
                torch.nn.utils.clip_grad_norm_(self.parameters(), 1.0)
                optimizer.step()
                epoch_loss += loss.item()

            avg_loss = epoch_loss/len(train_loader)
            losses.append(avg_loss)
            print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}")

        return losses

# ==================== 3. Initialize and Train ====================
# Sample data (replace with actual data)
n_users, n_items = 290, 300
train_data = np.random.choice([0,1,-1], size=(n_users, n_items), p=[0.3,0.2,0.5])
test_data = np.random.choice([0,1], size=(n_users, n_items))

# Create and train model
resips_model = ResIPS(n_users, n_items, latent_dim=64)
loss_history = resips_model.train_resips(
    train_data,
    val_data=test_data,
    alpha=0.5,
    beta=1.0,
    gamma=0.01,
    lr=0.001,
    epochs=10
)

Epoch 1/10, Loss: 11.4928
Epoch 2/10, Loss: 10.2022
Epoch 3/10, Loss: 9.4215
Epoch 4/10, Loss: 8.7182
Epoch 5/10, Loss: 8.0937
Epoch 6/10, Loss: 7.5025
Epoch 7/10, Loss: 6.9875
Epoch 8/10, Loss: 6.5269
Epoch 9/10, Loss: 6.1132
Epoch 10/10, Loss: 5.7415


In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

class ResidualLayer(nn.Module):
    """Residual network for propensity/imputation calibration"""
    def __init__(self, input_dim):
        super().__init__()
        self.residual = nn.Sequential(
            nn.Linear(input_dim, input_dim),
            nn.ReLU()
        )
    def forward(self, x):
        return x + self.residual(x)

class IPS(nn.Module):
    """Base IPS model with propensity estimation"""
    def __init__(self, n_users, n_items, latent_dim):
        super().__init__()
        self.user_emb = nn.Embedding(n_users, latent_dim)
        self.item_emb = nn.Embedding(n_items, latent_dim)
        self.propensity = nn.Sequential(
            nn.Linear(latent_dim*2, 1),
            nn.Sigmoid()
        )

    def forward(self, user, item):
        user_emb = self.user_emb(user)
        item_emb = self.item_emb(item)
        return torch.sigmoid((user_emb * item_emb).sum(dim=1))

In [ ]:
class ResIPS(IPS):
    """Res-IPS with calibrated propensities and imputations"""
    def __init__(self, n_users, n_items, latent_dim):
        super().__init__(n_users, n_items, latent_dim)
        # Residual networks
        self.residual_p = nn.Sequential(
            ResidualLayer(latent_dim*2),
            nn.Linear(latent_dim*2, 1)
        )
        self.residual_d = nn.Sequential(
            ResidualLayer(latent_dim*2),
            nn.Linear(latent_dim*2, 1)
        )
        # Imputation model
        self.imputation = nn.Sequential(
            nn.Linear(latent_dim*2, 1),
            nn.Sigmoid()
        )

    def _prepare_data(self, train_data, val_data):
        """Create data loaders for training (biased) and validation (unbiased)"""
        # Training data (observed interactions)
        train_users, train_items = np.where(train_data != -1)
        train_ratings = train_data[train_users, train_items]
        train_dataset = TensorDataset(
            torch.LongTensor(train_users),
            torch.LongTensor(train_items),
            torch.FloatTensor(train_ratings)
        )
        train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)

        # Validation data (unbiased)
        val_users, val_items = np.where(val_data != -1)
        val_ratings = val_data[val_users, val_items]
        val_dataset = TensorDataset(
            torch.LongTensor(val_users),
            torch.LongTensor(val_items),
            torch.FloatTensor(val_ratings)
        )
        val_loader = DataLoader(val_dataset, batch_size=128)

        return train_loader, val_loader

    def compute_loss(self, u, i, r, vu, vi, vr, alpha=0.5, beta=1.0, gamma=0.01, eps=1e-8):
        """Res-IPS Loss Calculation (from paper)"""
        # Feature embeddings
        user_emb = self.user_emb(u)
        item_emb = self.item_emb(i)
        phi = torch.cat([user_emb, item_emb], dim=1)

        # Propensity calibration
        p_nom = torch.clamp(self.propensity(phi).squeeze(), min=eps, max=1-eps)
        p_res = self.residual_p(phi).squeeze()
        p_tilde = torch.sigmoid(torch.logit(p_nom) + p_res).clamp(eps, 1-eps)

        # Imputation calibration
        delta_nom = self.imputation(phi).squeeze()
        delta_res = self.residual_d(phi).squeeze()
        delta_tilde = torch.sigmoid(torch.logit(delta_nom) + delta_res).clamp(eps, 1-eps)

        # Loss components
        pred = self(u, i)
        L_ctr = F.binary_cross_entropy(p_nom, r)  # CTR loss
        L_imp = F.mse_loss(delta_nom, (r - pred).detach())  # Imputation loss
        L_cvr_b = (F.binary_cross_entropy(pred, r, reduction='none') / p_tilde).mean()  # IPS term
        L_ctcvr_b = F.binary_cross_entropy(p_tilde * pred, r)  # CTCVR loss
        L_cvr_u = F.binary_cross_entropy(self(vu, vi), vr)  # Unbiased loss
        L_cons = F.mse_loss(p_nom, p_tilde) + F.mse_loss(delta_nom, delta_tilde)  # Consistency

        return L_ctr + alpha*L_imp + beta*(L_cvr_b + L_ctcvr_b + L_cvr_u) + gamma*L_cons

    def train_model(self, train_data, val_data, epochs=10, lr=0.001, **kwargs):
        """Training loop for Res-IPS"""
        optimizer = torch.optim.Adam(self.parameters(), lr=lr)
        train_loader, val_loader = self._prepare_data(train_data, val_data)
        losses = []

        for epoch in range(epochs):
            epoch_loss = 0.0
            for (u, i, r), (vu, vi, vr) in zip(train_loader, val_loader):
                loss = self.compute_loss(u, i, r, vu, vi, vr, **kwargs)
                optimizer.zero_grad()
                loss.backward()
                torch.nn.utils.clip_grad_norm_(self.parameters(), 1.0)
                optimizer.step()
                epoch_loss += loss.item()

            avg_loss = epoch_loss / len(train_loader)
            losses.append(avg_loss)
            print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}")

        return losses

In [ ]:
class ResDR(ResIPS):
    """Res-DR extends Res-IPS with Doubly Robust term"""
    def compute_loss(self, u, i, r, vu, vi, vr, alpha=0.5, beta=1.0, gamma=0.01, eps=1e-8):
        """Res-DR Loss Calculation (from paper)"""
        # Reuse Res-IPS components
        user_emb = self.user_emb(u)
        item_emb = self.item_emb(i)
        phi = torch.cat([user_emb, item_emb], dim=1)

        # Calibrated terms
        p_nom = torch.clamp(self.propensity(phi).squeeze(), min=eps, max=1-eps)
        p_res = self.residual_p(phi).squeeze()
        p_tilde = torch.sigmoid(torch.logit(p_nom) + p_res).clamp(eps, 1-eps)

        delta_nom = self.imputation(phi).squeeze()
        delta_res = self.residual_d(phi).squeeze()
        delta_tilde = torch.sigmoid(torch.logit(delta_nom) + delta_res).clamp(eps, 1-eps)

        # Compute base Res-IPS loss
        pred = self(u, i)
        L_ctr = F.binary_cross_entropy(p_nom, r)
        L_imp = F.mse_loss(delta_nom, (r - pred).detach())
        L_ctcvr_b = F.binary_cross_entropy(p_tilde * pred, r)
        L_cvr_u = F.binary_cross_entropy(self(vu, vi), vr)
        L_cons = F.mse_loss(p_nom, p_tilde) + F.mse_loss(delta_nom, delta_tilde)

        # Doubly Robust term (replace IPS with DR)
        delta = F.binary_cross_entropy(pred, r, reduction='none')
        L_dr_b = (delta_tilde + (delta - delta_tilde) / p_tilde).mean()

        return L_ctr + alpha*L_imp + beta*(L_dr_b + L_ctcvr_b + L_cvr_u) + gamma*L_cons

In [ ]:
# Initialize data
n_users, n_items = 290, 300
train_data = np.random.choice([0, 1, -1], size=(n_users, n_items), p=[0.3, 0.2, 0.5])  # Biased
test_data = np.random.choice([0, 1], size=(n_users, n_items))  # Unbiased


In [ ]:
#Train Res-IPS
resips = ResIPS(n_users, n_items, latent_dim=64)
resips_loss = resips.train_model(
    train_data, test_data,
    alpha=0.5, beta=1.0, gamma=0.01,
    lr=0.001, epochs=10
)

Epoch 1/10, Loss: 11.8021
Epoch 2/10, Loss: 10.3168
Epoch 3/10, Loss: 9.5094
Epoch 4/10, Loss: 8.7956
Epoch 5/10, Loss: 8.1624
Epoch 6/10, Loss: 7.5689
Epoch 7/10, Loss: 7.0252
Epoch 8/10, Loss: 6.5571
Epoch 9/10, Loss: 6.1221
Epoch 10/10, Loss: 5.7382


In [ ]:
# Train Res-DR
resdr = ResDR(n_users, n_items, latent_dim=64)
resdr_loss = resdr.train_model(
    train_data, test_data,
    alpha=0.5, beta=1.0, gamma=0.01,
    lr=0.001, epochs=10
)


Epoch 1/10, Loss: 11.7354
Epoch 2/10, Loss: 10.4335
Epoch 3/10, Loss: 9.6362
Epoch 4/10, Loss: 8.8784
Epoch 5/10, Loss: 8.2204
Epoch 6/10, Loss: 7.6223
Epoch 7/10, Loss: 7.0855
Epoch 8/10, Loss: 6.5779
Epoch 9/10, Loss: 6.1300
Epoch 10/10, Loss: 5.7449


In [ ]:
# ==================== 1. Revised Model Implementations ====================
class MF(nn.Module):
    """Matrix Factorization (Base Model)"""
    def __init__(self, n_users, n_items, latent_dim):
        super().__init__()
        self.user_emb = nn.Embedding(n_users, latent_dim)
        self.item_emb = nn.Embedding(n_items, latent_dim)

    def forward(self, user, item):
        user_emb = self.user_emb(user)
        item_emb = self.item_emb(item)
        return torch.sigmoid((user_emb * item_emb).sum(dim=1))

    def train_model(self, train_data, epochs=10, lr=0.001):
        optimizer = torch.optim.Adam(self.parameters(), lr=lr)
        criterion = nn.BCELoss()

        # Prepare data
        users, items = np.where(train_data != -1)
        ratings = train_data[users, items]
        dataset = TensorDataset(
            torch.LongTensor(users),
            torch.LongTensor(items),
            torch.FloatTensor(ratings)
        )
        loader = DataLoader(dataset, batch_size=128, shuffle=True)

        for epoch in range(epochs):
            for u, i, r in loader:
                pred = self(u, i)
                loss = criterion(pred, r)

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

class IPS(MF):
    """Inverse Propensity Scoring"""
    def __init__(self, n_users, n_items, latent_dim):
        super().__init__(n_users, n_items, latent_dim)
        self.propensity = nn.Sequential(
            nn.Linear(latent_dim*2, 1),
            nn.Sigmoid()
        )

    def train_model(self, train_data, epochs=10, lr=0.001):
        optimizer = torch.optim.Adam(self.parameters(), lr=lr)

        # Prepare data
        users, items = np.where(train_data != -1)
        ratings = train_data[users, items]
        dataset = TensorDataset(
            torch.LongTensor(users),
            torch.LongTensor(items),
            torch.FloatTensor(ratings)
        )
        loader = DataLoader(dataset, batch_size=128, shuffle=True)

        for epoch in range(epochs):
            for u, i, r in loader:
                # Get embeddings
                user_emb = self.user_emb(u)
                item_emb = self.item_emb(i)
                prop_input = torch.cat([user_emb, item_emb], dim=1)

                # Compute loss
                p = torch.clamp(self.propensity(prop_input).squeeze(), 1e-8, 1-1e-8)
                pred = self(u, i)
                loss = (F.binary_cross_entropy(pred, r, reduction='none') / p).mean()

                # Update
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

# Keep previous implementations for DR, ResIPS, ResDR unchanged
# [Use the DR, ResIPS, ResDR implementations from previous answer]

In [ ]:
# Initialize models
models = {
    "MF": MF(290, 300, 64),
    "IPS": IPS(290, 300, 64),
    "DR": DR(290, 300, 64),
    "Res-IPS": ResIPS(290, 300, 64),
    "Res-DR": ResDR(290, 300, 64)
}

# Train and evaluate
results = train_and_evaluate(
    models,
    train_data,
    test_data,
    epochs=10,
    k=5
)

# Print results
print("\n=== Final Results ===")
print(f"{'Model':<10} {'AUC':<8} {'Recall@5':<10} {'NDCG@5':<10}")
for model, metrics in results.items():
    print(f"{model:<10} {metrics['AUC']:<8} {metrics['Recall@5']:<10} {metrics['NDCG@5']:<10}")


=== Training MF ===

=== Training IPS ===

=== Training DR ===

=== Training Res-IPS ===
Epoch 1/10, Loss: 11.4889
Epoch 2/10, Loss: 10.2679
Epoch 3/10, Loss: 9.4557
Epoch 4/10, Loss: 8.7666
Epoch 5/10, Loss: 8.1238
Epoch 6/10, Loss: 7.5225
Epoch 7/10, Loss: 7.0109
Epoch 8/10, Loss: 6.5247
Epoch 9/10, Loss: 6.1140
Epoch 10/10, Loss: 5.7260

=== Training Res-DR ===
Epoch 1/10, Loss: 11.5682
Epoch 2/10, Loss: 10.2791
Epoch 3/10, Loss: 9.4699
Epoch 4/10, Loss: 8.7692
Epoch 5/10, Loss: 8.1088
Epoch 6/10, Loss: 7.5245
Epoch 7/10, Loss: 7.0002
Epoch 8/10, Loss: 6.5252
Epoch 9/10, Loss: 6.0790
Epoch 10/10, Loss: 5.6737

=== Final Results ===
Model      AUC      Recall@5   NDCG@5    
MF         0.499    0.49       0.524     
IPS        0.501    0.471      0.472     
DR         0.499    0.497      0.483     
Res-IPS    0.527    0.511      0.517     
Res-DR     0.524    0.514      0.556     
